Create a research and summarization agent using LangGraph. The agent should process user queries by determining whether they require reasoning from an LLM, web research, or retrieval from a knowledge base. The agent should leverage multiple specialized sub-agents to generate well-structured responses.

**The system should include:**
- **Router Agent:**  
  Determines whether a query should be answered using the LLM, web research, or retrieval-augmented generation (RAG).
- **Web Research Agent:**  
  If the query requires up-to-date information, this agent performs a web search and extracts relevant details.
- **RAG Agent:**  
  Handles queries related to a predefined dataset (e.g., any dataset that you have).
- **Summarization Agent:**  
  After gathering information, this agent synthesizes a final structured response.

In [0]:
!pip install -q langchain==0.3.14
!pip install -q langchain-openai==0.3.0
!pip install -q langchain-community==0.3.14
!pip install -q langgraph==0.2.64

In [0]:
!pip install ddgs

In [0]:
!pip install -q langchain-chroma==0.2.0

In [0]:
dbutils.library.restartPython()

In [0]:
import os
from typing import TypedDict, Annotated, List, Literal
from datetime import datetime
import operator

# LangGraph and LangChain imports
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.tools import tool
from langchain.docstore.document import Document
from tqdm import tqdm
import json
from langchain_chroma import Chroma
from ddgs import DDGS
from langgraph.checkpoint.memory import MemorySaver
# from langchain_chroma import Chroma

In [0]:
# Disable OpenTelemetry if causing issues
os.environ["OTEL_SDK_DISABLED"] = "true"

In [0]:
from getpass import getpass

OPENAI_KEY = getpass('Enter Open AI API Key: ')

In [0]:
os.environ['OPENAI_API_KEY'] = OPENAI_KEY

## State Definition

In [0]:
class AgentState(TypedDict):
    """
    Defines the state structure that flows through the agent graph.
    """
    query: str
    route_decision: str
    web_results: str
    rag_results: str
    llm_response: str
    final_summary: str
    messages: Annotated[List[str], operator.add]
    metadata: dict

In [0]:
llm = ChatOpenAI(model="gpt-4o", temperature=0)

## Create Knowledge Base

In [0]:
# # restaurant_knowledge_base.py

# """
# Food Factory Restaurant - Comprehensive Knowledge Base
# A modern fine-dining establishment with industrial-chic ambiance
# """

# FOOD_FACTORY_KNOWLEDGE_BASE = """

# === FOOD FACTORY - RESTAURANT OVERVIEW ===

# Food Factory is a premium fine-dining restaurant located in the heart of downtown Bangalore, India. 
# Established in 2019, Food Factory combines industrial-chic aesthetics with culinary excellence, 
# offering a unique dining experience that celebrates both traditional and contemporary cuisine. 
# The restaurant is housed in a renovated 1950s textile factory, preserving the original brick walls, 
# exposed metal beams, and vintage machinery while incorporating modern luxury elements.

# The restaurant spans 8,500 square feet across two levels, accommodating up to 120 guests in the main 
# dining area, with an additional private dining room for 24 guests and a rooftop bar that seats 40. 
# Food Factory has earned numerous accolades, including the Bangalore Fine Dining Award 2023, 
# recognition in India's Top 50 Restaurants 2024, and a prestigious AA Rosette for culinary excellence.


# === CULINARY PHILOSOPHY ===

# Food Factory's culinary philosophy centers on "Industrial Craftsmanship" - the meticulous transformation 
# of premium ingredients into edible art. Executive Chef Arjun Mehta, a Le Cordon Bleu graduate with 
# 15 years of international experience, leads a team of 12 skilled chefs who specialize in Modern Indian 
# Fusion cuisine with global influences.

# The restaurant's approach emphasizes:
# - Farm-to-table sourcing from local organic farms within 100 kilometers
# - Seasonal menu rotations every quarter to showcase fresh, peak-season ingredients
# - Molecular gastronomy techniques blended with traditional Indian cooking methods
# - Zero-waste kitchen practices with comprehensive composting and recycling programs
# - Artisanal preparation methods including in-house curing, smoking, and fermentation


# === SIGNATURE DISHES ===

# **Deconstructed Biryani Cloud**
# Price: ₹1,850
# A theatrical presentation of traditional Hyderabadi biryani reimagined. Slow-cooked lamb is served 
# with saffron-infused basmati rice air, crispy onion tuile, and a side of concentrated biryani essence. 
# The dish is finished tableside with aromatic smoke from burning bay leaves and cinnamon. 
# Preparation time: 48 hours for lamb marination and slow cooking.

# **Tandoori Octopus with Black Garlic Aioli**
# Price: ₹2,150
# Spanish octopus marinated in traditional tandoori spices, sous-vide for 4 hours, then charred in 
# the tandoor oven. Served with black garlic aioli, pickled radish, and microgreens. The dish showcases 
# the perfect marriage of Mediterranean seafood and Indian spice profiles.

# **Truffle Butter Chicken Ravioli**
# Price: ₹1,650
# House-made pasta filled with butter chicken reduction, topped with white truffle shavings, served 
# in a tomato-cream velouté. This dish represents Chef Arjun's signature fusion style, combining 
# Italian pasta-making techniques with India's most beloved curry.

# **Molecular Pani Puri Experience**
# Price: ₹950
# An innovative take on India's favorite street food. Six spherified pani puri waters in different 
# flavors (tamarind, mint, pomegranate, mango, jaljeera, and chocolate) served with crispy semolina 
# spheres and potato-chickpea filling. Each sphere bursts with flavor when consumed.

# **Miso-Glazed Tandoori Salmon**
# Price: ₹2,450
# Norwegian salmon marinated in white miso paste and tandoori spices, cooked in the tandoor, and 
# glazed with honey-miso reduction. Served with wasabi mashed potatoes and sautéed bok choy.

# **Saffron Panna Cotta with Rose Jalebi**
# Price: ₹750
# Silky saffron-infused Italian panna cotta topped with miniature rose-flavored jalebis, crushed 
# pistachios, and edible gold leaf. A perfect blend of Italian and Indian dessert traditions.


# === BEVERAGE PROGRAM ===

# Food Factory boasts an extensive beverage program curated by Master Sommelier Priya Sharma, 
# featuring over 300 wine labels from 15 countries, with a special focus on Indian wines from 
# Nashik and Bangalore regions.

# **Wine Collection:**
# - 120 Old World wines (France, Italy, Spain, Germany)
# - 100 New World wines (California, Australia, Chile, South Africa)
# - 80 Indian wines featuring boutique wineries
# - Temperature-controlled wine cellar maintaining optimal 55°F (13°C)
# - Wine pairing menus available for 5-course and 7-course tasting menus

# **Craft Cocktails:**
# The bar program features 25 signature cocktails created by mixologist Rohan Desai, each inspired 
# by Indian spices and ingredients:

# *Masala Old Fashioned* (₹850): Bourbon infused with cardamom and star anise, orange bitters, 
# demerara syrup, garnished with torched cinnamon stick.

# *Curry Leaf Martini* (₹800): Gin infused with curry leaves, dry vermouth, green chili tincture, 
# garnished with fresh curry leaf.

# *Saffron Negroni* (₹900): Saffron-infused gin, Campari, sweet vermouth, orange twist.

# *Tamarind Margarita* (₹750): Tequila, tamarind concentrate, lime juice, chili salt rim.

# **Non-Alcoholic Program:**
# 15 house-made mocktails, fresh-pressed juices, and artisanal sodas, all priced between ₹350-₹550.


# === DINING EXPERIENCES ===

# **Chef's Table Experience**
# Price: ₹8,500 per person (minimum 2 guests, maximum 6 guests)
# An exclusive 10-course tasting menu served at the kitchen counter, with each course personally 
# presented by Chef Arjun. Includes wine pairing, behind-the-scenes kitchen tour, and personalized 
# menu card signed by the chef. Duration: approximately 3.5 hours. Advance reservation required 
# (minimum 48 hours notice).

# **Tasting Menu - The Factory Journey**
# 5-Course Menu: ₹3,500 per person
# 7-Course Menu: ₹4,800 per person
# Wine Pairing: Additional ₹2,200 (5-course) or ₹3,200 (7-course)

# **Sunday Brunch**
# Time: 11:00 AM - 3:30 PM
# Price: ₹2,800 per person (includes unlimited mocktails)
# Price with alcohol: ₹3,800 per person (includes unlimited cocktails, wine, and beer)
# Features live cooking stations, seafood bar, dessert station, and live jazz music.

# **Private Dining Room**
# Capacity: 8-24 guests
# Rental: ₹25,000 (includes room setup, dedicated service staff)
# Customizable menus starting at ₹4,000 per person
# Ideal for corporate events, celebrations, and intimate gatherings.


# === AMBIANCE AND DESIGN ===

# Food Factory's interior design, created by renowned architect Kavita Patel, celebrates the building's 
# industrial heritage while incorporating contemporary luxury. Key design elements include:

# - Original exposed brick walls from the 1950s textile factory
# - Vintage industrial pendant lighting throughout the space
# - Custom-made tables crafted from reclaimed factory machinery
# - Open kitchen design allowing guests to observe culinary artistry
# - Floor-to-ceiling windows providing natural light during daytime
# - Curated art collection featuring local Bangalore artists
# - Acoustic design ensuring comfortable conversation despite high ceilings
# - Climate-controlled environment maintaining 22°C (72°F) year-round

# The rooftop bar, named "The Smokehouse," features:
# - Retractable glass roof for all-weather dining
# - Vertical herb garden supplying fresh herbs for cocktails
# - Wood-fired pizza oven for bar menu items
# - Panoramic views of Bangalore's skyline
# - DJ booth for weekend entertainment (Friday and Saturday, 8 PM - 12 AM)


# === OPERATIONAL DETAILS ===

# **Location and Contact:**
# Address: 42 Industrial Avenue, Indiranagar, Bangalore, Karnataka 560038, India
# Phone: +91-80-4567-8900
# Email: reservations@foodfactory.in
# Website: www.foodfactory.in

# **Operating Hours:**
# Monday - Thursday: 6:00 PM - 11:30 PM (Last order 10:45 PM)
# Friday - Saturday: 6:00 PM - 12:30 AM (Last order 11:45 PM)
# Sunday: 11:00 AM - 3:30 PM (Brunch), 6:00 PM - 11:00 PM (Dinner)
# Closed: Second Monday of every month for staff training

# **Reservation Policy:**
# - Reservations highly recommended, especially for weekends
# - Online booking available through website and OpenTable
# - Phone reservations accepted during business hours
# - Cancellation policy: 24-hour advance notice required
# - No-show fee: ₹1,000 per person for dinner reservations
# - Special occasions: Notify restaurant for birthday/anniversary celebrations

# **Dress Code:**
# Smart casual to formal attire. No shorts, flip-flops, or sleeveless shirts for gentlemen.
# Jackets not required but recommended for fine-dining experience.

# **Payment and Pricing:**
# - Average per person cost: ₹3,500 - ₹5,000 (excluding alcohol)
# - All major credit cards accepted (Visa, Mastercard, American Express)
# - Digital payments: Paytm, Google Pay, PhonePe accepted
# - Service charge: 10% added to bill
# - GST: 5% on food, 18% on alcohol (as per Indian tax regulations)
# - Gratuity: Not included, at guest's discretion


# === TEAM AND LEADERSHIP ===

# **Executive Chef Arjun Mehta**
# Le Cordon Bleu Paris graduate, previously worked at Noma (Copenhagen), Gaggan (Bangkok), and 
# Indian Accent (New Delhi). Specializes in Modern Indian Fusion cuisine with molecular gastronomy 
# techniques. Winner of Young Chef of the Year Award 2022.

# **Sous Chef Meera Krishnan**
# 15 years of experience in fine dining, trained at Culinary Institute of America. Oversees kitchen 
# operations and menu development. Expert in South Indian cuisine and vegetarian preparations.

# **Pastry Chef François Dubois**
# French-trained pastry chef with 12 years of experience. Previously worked at Le Bernardin (New York) 
# and Eleven Madison Park. Specializes in French-Indian fusion desserts.

# **Master Sommelier Priya Sharma**
# India's youngest female Master Sommelier, certified by Court of Master Sommeliers. Curates wine 
# list and conducts monthly wine tasting events. Expert in Indian wine industry.

# **General Manager Vikram Patel**
# 20 years of hospitality experience, including positions at Taj Hotels and Oberoi Group. 
# Oversees all front-of-house operations and guest experience.


# === SUSTAINABILITY INITIATIVES ===

# Food Factory is committed to environmental responsibility and sustainable practices:

# **Sourcing:**
# - 80% of ingredients sourced from local farms within 100 km radius
# - Direct partnerships with 25 organic farmers
# - Seasonal menu changes to reduce carbon footprint
# - Sustainable seafood certified by Marine Stewardship Council
# - Antibiotic-free meats from certified humane farms

# **Waste Management:**
# - Zero-waste kitchen policy with 95% waste diversion rate
# - Composting program processing 200 kg of organic waste monthly
# - Partnership with local urban farms for compost distribution
# - Food donation program with Feeding India NGO
# - Recycling program for glass, plastic, and metal

# **Energy and Water:**
# - Solar panels providing 40% of restaurant's energy needs
# - LED lighting throughout the facility
# - Water-efficient dishwashing and kitchen equipment
# - Rainwater harvesting system for landscaping irrigation
# - Energy-efficient HVAC system with smart temperature control


# === SPECIAL EVENTS AND PROGRAMS ===

# **Monthly Wine Dinners**
# First Thursday of every month, 7:00 PM
# Price: ₹4,500 per person
# Features guest winemakers, sommelier-led tastings, and specially paired 5-course menu.

# **Cooking Classes**
# Held every Saturday, 3:00 PM - 6:00 PM
# Price: ₹5,500 per person (includes recipes, ingredients, and dinner)
# Limited to 8 participants per class
# Learn to prepare three signature dishes with Chef Arjun or Sous Chef Meera.

# **Corporate Events and Buyouts**
# Full restaurant buyout: ₹5,00,000 (includes venue, basic setup, service staff)
# Capacity: 120 guests for seated dinner, 180 for cocktail reception
# Customizable menus and beverage packages available
# Dedicated event coordinator assigned for planning.

# **Seasonal Festivals**
# - Truffle Festival (October-November): Special truffle-focused menu
# - Monsoon Menu (June-September): Comfort food and regional specialties
# - Summer Grilling Series (March-May): Outdoor barbecue and grilled specialties
# - Holiday Season (December): Special festive menu and decorations


# === AWARDS AND RECOGNITION ===

# - Bangalore Fine Dining Award 2023 - Winner
# - India's Top 50 Restaurants 2024 - Ranked #17
# - AA Rosette for Culinary Excellence 2023
# - Best Wine Program in Bangalore 2023 - Sommelier India Awards
# - Sustainable Restaurant Award 2024 - Green Restaurant Association
# - TripAdvisor Travelers' Choice Award 2023, 2024
# - OpenTable Diners' Choice Award 2023, 2024
# - Featured in Condé Nast Traveler India - Best New Restaurants 2020
# - Featured in Food & Wine Magazine - Restaurants to Watch 2021
# - Chef Arjun Mehta - Young Chef of the Year 2022


# === CUSTOMER REVIEWS AND TESTIMONIALS ===

# "Food Factory is not just a meal, it's an experience. The Deconstructed Biryani Cloud was the most 
# innovative dish I've had in years. The theatrical presentation and explosive flavors make this a 
# must-visit for any food enthusiast." - Rajeev Masand, Food Critic, Times of India

# "The Chef's Table experience at Food Factory exceeded all expectations. Chef Arjun's passion for 
# food and his innovative approach to Indian cuisine is truly remarkable. Every course was a masterpiece." 
# - Anjali Verma, Verified OpenTable Diner

# "Exceptional wine program curated by Sommelier Priya. Her recommendations were spot-on, and the 
# pairing elevated our dining experience to another level. The Saffron Panna Cotta was divine!" 
# - Michael Thompson, Wine Enthusiast

# Average ratings:
# - Google Reviews: 4.7/5 (based on 1,245 reviews)
# - TripAdvisor: 4.5/5 (based on 892 reviews)
# - Zomato: 4.6/5 (based on 2,156 reviews)
# - OpenTable: 4.8/5 (based on 678 reviews)


# === FREQUENTLY ASKED QUESTIONS ===

# **Q: Do you accommodate dietary restrictions?**
# A: Yes, Food Factory accommodates all dietary restrictions including vegetarian, vegan, gluten-free, 
# dairy-free, nut allergies, and religious dietary requirements. Please inform us at the time of 
# reservation, and our chefs will prepare customized dishes.

# **Q: Is there parking available?**
# A: Yes, we offer complimentary valet parking for all guests. We also have a covered parking structure 
# with 50 spaces available on a first-come, first-served basis.

# **Q: Can I bring my own wine?**
# A: Yes, corkage fee is ₹1,500 per bottle (750ml) with a maximum of 2 bottles per table. 
# Not applicable on bottles already available in our wine list.

# **Q: Do you have a children's menu?**
# A: Yes, we offer a special children's menu for guests aged 12 and under, featuring kid-friendly 
# versions of our signature dishes, priced between ₹450-₹750 per dish.

# **Q: Is the restaurant wheelchair accessible?**
# A: Yes, Food Factory is fully wheelchair accessible with ramps, accessible restrooms, and 
# designated seating areas. Please inform us at reservation if you require special accommodations.

# **Q: Do you offer gift cards?**
# A: Yes, gift cards are available in denominations of ₹2,000, ₹5,000, and ₹10,000. They can be 
# purchased online or at the restaurant and are valid for 12 months from the date of purchase.

# **Q: Can I host a private event at Food Factory?**
# A: Yes, we have a private dining room that accommodates 8-24 guests, and we also offer full 
# restaurant buyouts for larger events. Please contact our events team at events@foodfactory.in 
# for more information.

# **Q: Do you take walk-ins?**
# A: While we accept walk-ins based on availability, we highly recommend making a reservation, 
# especially for weekend dining, as we often operate at full capacity.


# === CONTACT INFORMATION FOR SPECIFIC INQUIRIES ===

# General Reservations: reservations@foodfactory.in | +91-80-4567-8900
# Private Events: events@foodfactory.in | +91-80-4567-8901
# Chef's Table Bookings: chefstable@foodfactory.in | +91-80-4567-8902
# Corporate Inquiries: corporate@foodfactory.in | +91-80-4567-8903
# Press and Media: media@foodfactory.in | +91-80-4567-8904
# Gift Cards: giftcards@foodfactory.in | +91-80-4567-8905

# Social Media:
# Instagram: @foodfactory_bangalore (125K followers)
# Facebook: facebook.com/foodfactorybangalore (85K likes)
# Twitter: @FoodFactoryBLR (42K followers)
# YouTube: Food Factory Bangalore (cooking videos and behind-the-scenes content)

# """

# # Additional structured data for enhanced RAG responses
# MENU_DATA = {
#     "appetizers": [
#         {
#             "name": "Molecular Pani Puri Experience",
#             "price": 950,
#             "category": "Vegetarian",
#             "description": "Six spherified pani puri waters with crispy semolina spheres",
#             "preparation_time": "15 minutes",
#             "allergens": ["gluten", "chickpeas"]
#         },
#         {
#             "name": "Tandoori Octopus with Black Garlic Aioli",
#             "price": 2150,
#             "category": "Seafood",
#             "description": "Spanish octopus marinated in tandoori spices, charred in tandoor",
#             "preparation_time": "4+ hours",
#             "allergens": ["seafood", "garlic"]
#         },
#         {
#             "name": "Truffle Burrata with Beetroot Carpaccio",
#             "price": 1450,
#             "category": "Vegetarian",
#             "description": "Imported burrata cheese with truffle oil and roasted beetroot",
#             "preparation_time": "10 minutes",
#             "allergens": ["dairy"]
#         }
#     ],
#     "main_courses": [
#         {
#             "name": "Deconstructed Biryani Cloud",
#             "price": 1850,
#             "category": "Lamb",
#             "description": "Slow-cooked lamb with saffron rice air and biryani essence",
#             "preparation_time": "48 hours",
#             "allergens": ["none"]
#         },
#         {
#             "name": "Truffle Butter Chicken Ravioli",
#             "price": 1650,
#             "category": "Poultry",
#             "description": "House-made pasta with butter chicken filling and truffle",
#             "preparation_time": "2 hours",
#             "allergens": ["gluten", "dairy", "eggs"]
#         },
#         {
#             "name": "Miso-Glazed Tandoori Salmon",
#             "price": 2450,
#             "category": "Seafood",
#             "description": "Norwegian salmon with miso-tandoori marinade",
#             "preparation_time": "45 minutes",
#             "allergens": ["seafood", "soy"]
#         }
#     ],
#     "desserts": [
#         {
#             "name": "Saffron Panna Cotta with Rose Jalebi",
#             "price": 750,
#             "category": "Dessert",
#             "description": "Italian panna cotta with Indian jalebi fusion",
#             "preparation_time": "4+ hours",
#             "allergens": ["dairy"]
#         },
#         {
#             "name": "Chocolate Samosa with Vanilla Ice Cream",
#             "price": 650,
#             "category": "Dessert",
#             "description": "Crispy chocolate-filled samosa with house-made ice cream",
#             "preparation_time": "20 minutes",
#             "allergens": ["gluten", "dairy", "nuts"]
#         }
#     ]
# }

# RESERVATION_INFO = {
#     "peak_hours": ["Friday 7-9 PM", "Saturday 7-9 PM", "Sunday Brunch 12-2 PM"],
#     "advance_booking_recommended": "2-3 weeks for weekends, 1 week for weekdays",
#     "cancellation_policy": "24 hours notice required",
#     "deposit_required": "For groups of 8 or more: ₹1,000 per person",
#     "special_occasions": "Complimentary dessert for birthdays/anniversaries with advance notice"
# }



In [0]:
# # def create_food_factory_vectorstore():
# """
# Creates a vector store specifically for Food Factory restaurant data
# """

# # Combine all knowledge base content
# full_content = FOOD_FACTORY_KNOWLEDGE_BASE

# # Add structured data as formatted text
# full_content += "\n\n=== MENU DATA (STRUCTURED) ===\n"
# full_content += json.dumps(MENU_DATA, indent=2)

# full_content += "\n\n=== RESERVATION INFORMATION (STRUCTURED) ===\n"
# full_content += json.dumps(RESERVATION_INFO, indent=2)

# # Split into chunks
# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=1000,
#     chunk_overlap=150,
#     length_function=len,
#     separators=["\n\n===", "\n\n", "\n", " ", ""]
# )

# chunks = text_splitter.split_text(full_content)

# print(f"Created {len(chunks)} chunks from Food Factory knowledge base")

# # Create embeddings and vector store
# embeddings = OpenAIEmbeddings(model='text-embedding-3-small')
# # vectorstore = FAISS.from_texts(chunks, embeddings)

# vectorstore = Chroma.from_texts(texts=chunks,
#                                 collection_name='knowledge_base',
#                                 embedding=embeddings,
#                                 # need to set the distance function to cosine else it uses euclidean by default
#                                 # check https://docs.trychroma.com/guides#changing-the-distance-function
#                                 collection_metadata={"hnsw:space": "cosine"},
#                                 persist_directory="./databricks_knowledge_base")

# # Save for later use
# # vectorstore.save_local("./food_factory_vectorstore")

# # print("Food Factory vector store created and saved successfully!")

# # return vectorstore

## Create RAG Agent

In [0]:
def restaurant_rag_agent(state: AgentState) -> AgentState:
    """
    Restaurant-specific RAG Agent using ChromaDB
    """

    query = state["query"]
    try:
        embeddings = OpenAIEmbeddings(model='text-embedding-3-small')
        vs = Chroma(
            collection_name='knowledge_base',
            embedding_function=embeddings,
            persist_directory="./databricks_knowledge_base"
        )
        vectorSearch = vs.as_retriever(search_type="similarity_score_threshold",
                                            search_kwargs={"k": 3, "score_threshold": 0.2})

        relevant_docs = vectorSearch.invoke(query)
        # result = vs.similarity_search(query)

        if not relevant_docs:
            state["rag_results"] = "I apologize, but I couldn't find relevant information in our knowledge base to answer your query. Please contact the restaurant directly at +91-80-4567-8900 or reservations@foodfactory.in for assistance."
            state["messages"].append("No relevant documents found")
            state["metadata"]["rag_docs_count"] = 0
            state["metadata"]["rag_timestamp"] = datetime.now().isoformat()
            print("⚠️ No relevant documents found")
            return state    


        # Combine retrieved documents
        context = "\n\n".join([doc.page_content for doc in relevant_docs])

        # Enhanced prompt for restaurant queries
        rag_prompt = f"""
        You are a knowledgeable assistant for Food Factory, a fine-dining restaurant in Bangalore.
        Using the following information from the restaurant's knowledge base, provide a comprehensive 
        and helpful answer to the customer's query.

        Context from Food Factory Knowledge Base:
        {context}

        Customer Query: {query}

        Provide a detailed, friendly, and professional response that:
        1. Directly answers the customer's question
        2. Includes specific details like prices, times, or requirements when relevant
        3. Suggests related information that might be helpful
        4. Maintains a welcoming and enthusiastic tone about the restaurant
        5. Includes contact information or booking instructions when appropriate

        If the context doesn't contain enough information to fully answer the query, 
        mention what is available and suggest contacting the restaurant directly.
        """
        messages = [
                    SystemMessage(content="You are a helpful Food Factory restaurant assistant."),
                    HumanMessage(content=rag_prompt)
                ]
                
        response = llm.invoke(messages)
        rag_answer = response.content
        state["rag_results"] = rag_answer
        state["messages"].append(f"Restaurant RAG retrieval: {len(relevant_docs)} documents")
        if "metadata" not in state or state["metadata"] is None:
            state["metadata"] = {}
        state["metadata"]["rag_docs_count"] = len(relevant_docs)
        state["metadata"]["rag_timestamp"] = datetime.now().isoformat()
        
        print(f"✅ Restaurant information retrieved: {len(relevant_docs)} documents")
        
    except Exception as e:
        import traceback
        error_msg = f"Restaurant RAG error: {str(e)}"
        error_details = traceback.format_exc()
        
        state["rag_results"] = f"I apologize, but I encountered an error while retrieving information. Please try again or contact us directly."
        state["messages"].append(error_msg)
        
        # Safely update metadata
        if "metadata" not in state or state["metadata"] is None:
            state["metadata"] = {}
        
        state["metadata"]["error"] = error_msg
        state["metadata"]["error_timestamp"] = datetime.now().isoformat()
        
        print(f"❌ {error_msg}")
        print(f"Details: {error_details}")
    
    return state

In [0]:
# Initialize the graph
workflow = StateGraph(AgentState)

# Add nodes for each agent
workflow.add_node("rag_agent", restaurant_rag_agent)

workflow.add_edge("rag_agent", END)

# Set entry point
workflow.set_entry_point("rag_agent")

# Compile the graph
agent = workflow.compile()

In [0]:
from IPython.display import display, Image, Markdown

display(Image(agent.get_graph().draw_mermaid_png()))

In [0]:
def call_agent(agent, prompt, user_session_id, verbose=False):
    events = agent.stream(
        {"query": prompt}, # initial state of the agent
        {"configurable": {"thread_id": user_session_id}},
        stream_mode="values",
    )

    print('Running Agent. Please wait...')
    for event in events:
        if verbose:
                print(event)

    display(Markdown(event['rag_results']))

In [0]:
user_session_id = "u01"
call_agent(agent,"Tell me about your Chef's Table experience", user_session_id, verbose=True)

## Create Web Search Agent

In [0]:
def web_search_agent(state: AgentState) -> AgentState:
    """
    Web Search Agent using DuckDuckGo
    """
    query = state["query"]
    try:
        ddgs = DDGS()
        search_results = ddgs.text(query, max_results=10)
        # Enhanced prompt for information extraction
        extraction_prompt = f"""
        Based on the following web search results, extract and organize the most relevant 
        and accurate information related to the query: "{query}"
        
        Search Results:
        {search_results}
        
        Provide a well-structured summary with:
        1. Key findings (3-5 bullet points)
        2. Important details and context
        3. Sources or references if mentioned
        
        Be concise but comprehensive.
        """
        messages = [
            SystemMessage(content="You are a web research analyst extracting key information."),
            HumanMessage(content=extraction_prompt)
        ]
        response = llm.invoke(messages)
        extracted_info = response.content
        
        state["web_results"] = extracted_info
        state["messages"].append(f"Web Research completed: {len(extracted_info)} characters")
        # Safely update metadata
        if "metadata" not in state or state["metadata"] is None:
            state["metadata"] = {}
        state["metadata"]["web_search_timestamp"] = datetime.now().isoformat()
        
        print(f"✅ Web Research completed: {len(extracted_info)} characters retrieved")
        
    except Exception as e:
        error_msg = f"Web search error: {str(e)}"
        state["web_results"] = error_msg
        state["messages"].append(error_msg)
        print(f"❌ {error_msg}")
    
    return state

In [0]:
# Initialize the graph
workflow = StateGraph(AgentState)

# Add nodes for each agent
workflow.add_node("web_search_agent", web_search_agent)

workflow.add_edge("web_search_agent", END)

# Set entry point
workflow.set_entry_point("web_search_agent")

# Compile the graph
agent = workflow.compile()

In [0]:
from IPython.display import display, Image, Markdown

display(Image(agent.get_graph().draw_mermaid_png()))

In [0]:
def call_agent(agent, prompt, user_session_id, verbose=False):
    events = agent.stream(
        {"query": prompt}, # initial state of the agent
        {"configurable": {"thread_id": user_session_id}},
        stream_mode="values",
    )

    print('Running Agent. Please wait...')
    for event in events:
        if verbose:
                print(event)

    display(Markdown(event['web_results']))

In [0]:
user_session_id = "u01"
call_agent(agent,"what is the PE ratio of Nvidia?", user_session_id, verbose=True)

## Create LLM Agent

In [0]:
def llm_agent(state: AgentState) -> AgentState:
    """
    LLM Agent: Handles general queries using direct LLM reasoning.
    Used for creative tasks, general knowledge, and reasoning queries.
    """
    query = state["query"]
    
    print(f"\n🤖 LLM Agent: Processing '{query}'")
    
    try:
        llm_prompt = f"""
        You are a helpful AI assistant with expertise in technology, data engineering, 
        and general knowledge. Provide a comprehensive and well-structured response 
        to the following query.
        
        Query: {query}

        Sample Query:
        1.Write Python Function for Prime Number.
        2. What is Data Engineering.
        
        Provide a detailed, accurate, and helpful response. Use examples where appropriate.
        """
        
        messages = [
            SystemMessage(content="You are an expert AI assistant."),
            HumanMessage(content=llm_prompt)
        ]
        
        response = llm.invoke(messages)
        llm_answer = response.content
        
        state["llm_response"] = llm_answer
        state["messages"].append(f"LLM response generated: {len(llm_answer)} characters")
        # Safely update metadata
        if "metadata" not in state or state["metadata"] is None:
            state["metadata"] = {}
        state["metadata"]["llm_timestamp"] = datetime.now().isoformat()
        
        print(f"✅ LLM response generated: {len(llm_answer)} characters")
        
    except Exception as e:
        error_msg = f"LLM error: {str(e)}"
        state["llm_response"] = error_msg
        state["messages"].append(error_msg)
        print(f"❌ {error_msg}")
    
    return state

## Create Router AGent

In [0]:
def router_agent(state: AgentState) -> AgentState:
    """
    Router Agent: Determines the appropriate path for query processing.
    Routes to: Web Research, RAG, or Direct LLM response.
    """
    query = state["query"]
    
    # Keywords that indicate need for current/live information
    web_keywords = [
        "latest", "current", "recent", "today", "now", "2024", "2025",
        "breaking", "news", "update", "trending", "live"
    ]
    
    # Keywords that indicate knowledge base queries
    rag_keywords = [
        "FOOD FACTORY - RESTAURANT", "SIGNATURE DISHES", "SIGNATURE DISHES","BEVERAGE PROGRAM","DINING EXPERIENCES", "restaurant","Dishes","restaurant",
        "menu","food","Operating Hours"
    ]
    
    routing_prompt = f"""
    You are a routing agent. Analyze the following query and determine the best processing path.
    
    Query: "{query}"
    
    Choose ONE of the following routes:
    - "web_research": If the query asks about current events, latest news, or time-sensitive information
    - "rag": If the query is about FOOD FACTORY - RESTAURANT.
    - "llm": If the query is a general question, creative task, write some code or requires reasoning without specific data
    
    Respond with ONLY the route name (web_research, rag, or llm).
    """
    
    messages = [
        SystemMessage(content="You are a query routing specialist."),
        HumanMessage(content=routing_prompt)
    ]
    
    response = llm.invoke(messages)
    route = response.content.strip().lower()
    
    # Fallback logic with keyword matching
    if route not in ["web_research", "rag", "llm"]:
        query_lower = query.lower()
        if any(keyword in query_lower for keyword in web_keywords):
            route = "web_research"
        elif any(keyword in query_lower for keyword in rag_keywords):
            route = "rag"
        else:
            route = "llm"
    
    state["route_decision"] = route
    state["messages"].append(f"Router Decision: {route}")
    # Safely update metadata
    if "metadata" not in state or state["metadata"] is None:
        state["metadata"] = {}
    state["metadata"]["routing_timestamp"] = datetime.now().isoformat()
    
    print(f"\n🔀 Router Agent: Routing to '{route}'")
    
    return state

## Create Summarization Agent

In [0]:
def summarization_agent(state: AgentState) -> AgentState:
    """
    Summarization Agent: Synthesizes information from all sources into a 
    coherent, well-structured final response.
    """
    query = state["query"]
    route = state["route_decision"]
    
    print(f"\n📝 Summarization Agent: Creating final summary")
    
    # Gather all available information
    information_sources = []
    
    if state.get("web_results"):
        information_sources.append(f"Web Research Results:\n{state['web_results']}")
    
    if state.get("rag_results"):
        information_sources.append(f"Knowledge Base Results:\n{state['rag_results']}")
    
    if state.get("llm_response"):
        information_sources.append(f"LLM Response:\n{state['llm_response']}")
    
    combined_info = "\n\n---\n\n".join(information_sources)
    
    summarization_prompt = f"""
    You are a summarization specialist. Create a comprehensive, well-structured final response 
    by synthesizing the following information sources.
    
    Original Query: {query}
    Processing Route: {route}
    
    Information Sources:
    {combined_info}
    
    Create a final response that:
    1. Directly answers the user's query
    2. Integrates information from all sources coherently
    3. Is well-structured with clear sections
    4. Includes relevant details and examples
    5. Maintains accuracy and clarity
    6. Ends with a brief summary or key takeaways if appropriate
    
    Format the response professionally with proper markdown formatting.
    """
    
    messages = [
        SystemMessage(content="You are an expert at synthesizing information into clear, comprehensive responses."),
        HumanMessage(content=summarization_prompt)
    ]
    
    response = llm.invoke(messages)
    final_summary = response.content
    
    state["final_summary"] = final_summary
    state["messages"].append("Final summary generated")
    # Safely update metadata
    if "metadata" not in state or state["metadata"] is None:
        state["metadata"] = {}
    state["metadata"]["summary_timestamp"] = datetime.now().isoformat()
    state["metadata"]["summary_length"] = len(final_summary)
    
    print(f"✅ Final summary generated: {len(final_summary)} characters")
    
    return state

In [0]:
# ==================== Routing Logic ====================

def route_query(state: AgentState) -> Literal["web_research", "rag", "llm"]:
    """
    Conditional edge function that determines which agent to call next.
    """
    return state["route_decision"]

In [0]:
"""
Creates and compiles the LangGraph workflow with all agents and routing logic.
"""

# Initialize the graph
workflow = StateGraph(AgentState)

# Add nodes for each agent
workflow.add_node("router", router_agent)
workflow.add_node("web_research", web_search_agent)
workflow.add_node("rag", restaurant_rag_agent)
workflow.add_node("llm", llm_agent)
workflow.add_node("summarizer", summarization_agent)

# Set entry point
workflow.set_entry_point("router")

# Add conditional edges from router to specialized agents
workflow.add_conditional_edges(
    "router",
    route_query,
    {
        "web_research": "web_research",
        "rag": "rag",
        "llm": "llm"
    }
)

# All specialized agents flow to summarizer
workflow.add_edge("web_research", "summarizer")
workflow.add_edge("rag", "summarizer")
workflow.add_edge("llm", "summarizer")

# Summarizer is the final node
workflow.add_edge("summarizer", END)

memory = MemorySaver()

# Compile the graph
research_agent = workflow.compile(checkpointer=memory)

In [0]:
def call_agent(agent, prompt, user_session_id, verbose=False):
    events = agent.stream(
        {"query": prompt}, # initial state of the agent
        {"configurable": {"thread_id": user_session_id}},
        stream_mode="values",
    )

    print('Running Agent. Please wait...')
    for event in events:
        if verbose:
                print(event)

    display(Markdown(event['final_summary']))

In [0]:
from IPython.display import display, Image, Markdown

display(Image(research_agent.get_graph(xray=True).draw_mermaid_png()))

## Question is for some Latest News. So our Agent would use Web_Search Subagent.

In [0]:
user_session_id = "u01"
call_agent(research_agent,"what is the PE ratio of Nvidia?", user_session_id, verbose=True)


## Question is for some Generic Coding Question. So our Agent would use Web_Search Subagent.

In [0]:
user_session_id = "u01"
call_agent(research_agent,"Write a Python function to calculate fibonacci numbers", user_session_id, verbose=True)

## RAG Agent in Action

In [0]:
user_session_id = "u01"
call_agent(research_agent,"Can I host a private event for 20 people in Food Factory Restaurant", user_session_id, verbose=True)

In [0]:
call_agent(research_agent,"What about private event with 40 people", user_session_id, verbose=True)

In [0]:
user_session_id = "u01"
call_agent(research_agent,"what about 60 people event", user_session_id, verbose=True)

In [0]:
user_session_id = "u02"
call_agent(research_agent,"what about 40 people", user_session_id, verbose=True)

In [0]:
user_session_id = "u02"
call_agent(research_agent,"what about 60 people event", user_session_id, verbose=True)